# Prompt Declaration Language

Prompt engineering is difficult: minor variations in prompts have large impacts on the output of LLMs and prompts are model-dependent. In recent years <i> prompt programming languages </i> have emerged to bring discipline to prompt engineering. Many of them are embedded in an imperative language such as Python or TypeScript, making it difficult for users to directly interact with prompts and multi-turn LLM interactions.

The Prompt Declaration Language (PDL) is a yaml-based declarative approach to prompt programming, where prompts are at the forefront. PDL facilitates model chaining and tool use, abstracting away the plumbing necessary for such compositions, enables type checking of the input and output of models, and is based on LiteLLM to support a variety of model providers. PDL has been used with RAG, CoT, ReAct, and an agent for solving SWE-bench. PDL is [open-source](https://github.com/IBM/prompt-declaration-language) and works well with watsonx.ai and Granite models.

All examples in this notebook use the new ibm/granite-8b-instruct-preview-4k model. You can use PDL stand-alone or from a Python SDK or, as shown here, in a notebook via a notebook extension. In the cell output, model-generated text is rendered in green font, and tool-generated text is rendered in purple font.

In [4]:
! pip install prompt-declaration-language
! pip install 'prompt-declaration-language[examples]'

In [2]:
%load_ext pdl.pdl_notebook_ext

## Model call

In PDL, prompts are the forefront and the user specifies step-by-step the shape of data they want to generate. In the following, the `text` construct indicates a text block containing a prompt and a model call. Implicitly PDL builds a background conversational context (list of role/content) which is used to make model calls. Each model call uses the context built so far as its input prompt.

In [2]:
%%pdl --reset-context
text: 
- "What is the meaning of life?\n"
- model: ibm/granite-8b-instruct-preview-4k
  platform: bam

What is the meaning of life?
The meaning of life is a philosophical and metaphysical question related to the purpose or significance of life or existence in general. This concept has been approached by many perspectives including philosophy, religion, and science. Some people find meaning through personal growth, relationships, love, and through helping others. Others seek meaning through spirituality or religious beliefs. Ultimately, the meaning of life may be a personal and subjective experience.

## Model chaining
Model chaining can be done by simply adding to the list of models to call declaratively. Since this cell has the `%%pdl` cell magic without `--reset-context`, it executes in the context created by the previous cell.

In [3]:
%%pdl
text:
- "\nSay it like a poem\n"
- model: ibm/granite-8b-instruct-preview-4k
  platform: bam
- "\n\nTranslate it to French\n"
- model: ibm/granite-8b-instruct-preview-4k
  platform: bam


Say it like a poem
Life's meaning, a question vast,
In philosophy, religion, and science cast.
Some find purpose in personal growth,
In love and relationships, they find their troth.
Others seek meaning through spirituality,
In faith and belief, they find their reality.
Ultimately, meaning is a personal quest,
In life's journey, we are put to the test.

Translate it to French
La signification de la vie, une question immense,
Dans la philosophie, la religion et la science, elle est posée.
Quelques-uns trouvent un but dans leur croissance personnelle,
Dans l'amour et les relations, ils trouvent leur foi.
D'autres cherchent la signification à travers la spiritualité,
Dans la foi et la croyance, ils trouvent leur réalité.
En fin de compte, le but est une quête personnelle,
Dans le voyage de la vie, nous sommes mis à l'épreuve.

## Chat templates

The second call to the model in the above program submits the following prompt. PDL takes care of applying the appropriate chat templates and tags, and builds the background context implicitly. Chat templates make your program easier to port across models, since you do not need to specify control tokens by hand. All the user has to do is list the models they want to chain, PDL takes care of the rest.

```
<|start_of_role|>user<|end_of_role|>What is the meaning of life?
<|end_of_text|>
The meaning of life is a philosophical and metaphysical question related to the purpose or significance of life or existence in general. This concept has been approached by many perspectives including philosophy, religion, and science. Some people find meaning through personal growth, relationships, love, and through helping others. Others seek meaning through spirituality or religious beliefs. Ultimately, the meaning of life may be a personal and subjective experience.

<|start_of_role|>user<|end_of_role|>Say it like a poem<|end_of_text|>
Life's meaning, a question vast,
In philosophy, religion, and science cast.
Some find purpose in personal growth,
In love and relationships, they find their troth.
Others seek meaning through spirituality,
In faith and belief, they find their reality.
Ultimately, meaning is a personal quest,
In life's journey, we are put to the test.

<|start_of_role|>user<|end_of_role|>Translate it to French
<|end_of_text|>
<|start_of_role|>assistant<|end_of_role|>
```

## Data pipeline

The following program shows a common prompting pattern: read some data, formulate a prompt using that data, submit to a model, and evaluate. In this program, we formulate a prompt for code explanation. The program first defines two variables: `code`, which holds the data we read, and `truth` for the ground truth. It then prints out the source code, formulates a prompts with the data, and calls a model to get an explanation. Finally, a Python code block is used to use the Levenshtein text distance metric and evaluate the explanation against the ground truth. This pipeline can similarly be applied to an entire data set to produce a jsonl file.

In [4]:
%%pdl
defs:
  code:
    read: ./data.yaml
    parser: yaml
  truth:
    read: ./ground_truth.txt
text:
- "\n${ code.source_code }\n"
- model: ibm/granite-8b-instruct-preview-4k
  platform: bam
  def: explanation
  input: |
      Here is some info about the location of the function in the repo.
      repo: 
      ${ code.repo_info.repo }
      path: ${ code.repo_info.path }
      Function_name: ${ code.repo_info.function_name }


      Explain the following code:
      ```
      ${ code.source_code }```
- |

  Evaluation:
  The similarity (Levenshtein) between this answer and the ground truth is:
- def: EVAL
  lang: python
  code: |
    import textdistance
    expl = """
    ${ explanation }
    """
    truth = """
    ${ truth }
    """
    result = textdistance.levenshtein.normalized_similarity(expl, truth)



@SuppressWarnings("unchecked")
public static Map<String, String> deserializeOffsetMap(String lastSourceOffset) throws IOException {
  Map<String, String> offsetMap;
  if (lastSourceOffset == null || lastSourceOffset.isEmpty()) {    
    offsetMap = new HashMap<>();  
  } else {
    offsetMap = JSON_MAPPER.readValue(lastSourceOffset, Map.class);  
  }
  return offsetMap;
}


This Java method, `deserializeOffsetMap`, is designed to convert a JSON string into a `Map<String, String>`. Here's a breakdown of the code:

1. The method takes a single parameter, `lastSourceOffset`, which is expected to be a JSON string representing a map.

2. It first checks if `lastSourceOffset` is either null or an empty string. If true, it initializes a new `HashMap` and assigns it to `offsetMap`.

3. If `lastSourceOffset` is not null or empty, it uses Jackson's `JSON_MAPPER` to deserialize the JSON string into a `Map<String, String>` and assigns it to `offsetMap`.

4. Finally, it returns the `offsetMap`.

T

## Agentic Flow

The following PDL program shows an agentic flow with a ReAct prompt pattern. It first reads some demonstrations to be used as few-shots. The ReAct pattern is captured with PDL control structures (repeat-until, and if-then-else), and consists of cycling through thoughts, actions, and observations. The tools available are wikipedia search, and calculator (as a Python code). The agent decides when to search and when to calculate. The `spec` indicates a type for the output of the model when actions are produced, it is used to dynamically check outputs of models and fail when they don't conform to the expectation. 

In [16]:
%%pdl --reset-context
text:
- read: demonstrations.txt
  contribute: [context]
- "How many years ago was the discoverer of the Hudson River born? Keep in mind we are in 2024.\n"
- repeat:
    text:
    - def: thought
      model: ibm/granite-8b-instruct-preview-4k
      platform: bam
      parameters:
        stop_sequences: ["Act:"]
        include_stop_sequence: true
    - def: action
      model: ibm/granite-8b-instruct-preview-4k
      platform: bam
      parameters:
        stop_sequences: ["\n"]
      parser: json
      spec: {name: str, arguments: obj}
    - def: observation
      if: ${ action.name == "Search" }
      then:
        text:
        - "Obs: "
        - lang: python
          code: |
            import warnings, wikipedia
            warnings.simplefilter("ignore")
            try:
              result = wikipedia.summary("${ action.arguments.topic }", auto_suggest=False)
            except wikipedia.WikipediaException as e:
              result = str(e)
        - "\n"
      else:
        - if: ${ action.name == "Calc" }
          then:
            text:
            - "Obs: "
            - lang: python
              code: result = ${ action.arguments.expr }
            - "\n"
  until: ${ action.name != "Search" }

How many years ago was the discoverer of the Hudson River born? Keep in mind we are in 2024.
Act:{"name": "Search", "arguments": {"topic": "Henry Hudson"}}
Obs: Henry Hudson (c. 1565 – disappeared 23 June 1611) was an English sea explorer and navigator during the early 17th century, best known for his explorations of present-day Canada and parts of the Northeastern United States.
In 1607 and 1608, Hudson made two attempts on behalf of English merchants to find a rumoured Northeast Passage to Cathay via a route above the Arctic Circle. In 1609, he landed in North America on behalf of the Dutch East India Company and explored the region around the modern New York metropolitan area. Looking for a Northwest Passage to Asia on his ship Halve Maen ("Half Moon"), he sailed up the Hudson River, which was later named after him, and thereby laid the foundation for Dutch colonization of the region. His contributions to the exploration of the New World were significant and lasting. His voyages hel

## Conclusion

Since prompts are at the forefront, PDL makes users more productive in their trial-and-error with LLMs. PDL is a representation for prompting that can be used not only for low-level prompting, but as